In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import PIL
from torch import nn,optim
import torchvision 
from torchvision import transforms
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [2]:
train_imgs=torchvision.datasets.ImageFolder(r'F:\study\ml\HotDog\train')
test_imgs=torchvision.datasets.ImageFolder(r'F:\study\ml\HotDog\test')

In [3]:
def show_images(imgs,num_rows,num_cols,scale=1.5):
    figsize=(num_cols*scale,num_rows*scale)
    plt.subplots(num_rows,num_cols,figsize=figsize)
    for i in range(num_cols*num_rows):
        plt.subplot(num_rows,num_cols,i+1)
        if isinstance(imgs[i],PIL.Image.Image):
            plt.imshow(np.array(imgs[i]))
        else:
            plt.imshow(imgs[i])
    plt.show()
            

In [4]:
hotdogs=[train_imgs[i][0] for i in range(8)]
not_hotdogs=[train_imgs[-i-1][0] for i in range(8)]
show_images(hotdogs+not_hotdogs,2,8,1.5)

In [5]:
normize=torchvision.transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])

In [6]:
train_augs = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), normize
])

In [7]:
test_augs=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normize
])

In [8]:
pretrained_net=torchvision.models.resnet18(pretrained=True)

In [9]:
for i in pretrained_net.named_parameters():
    print(i[0])

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.weight
layer2.1.bn2.weight
layer2.1.bn2.bias
layer3.0.conv1.weight
layer3.0.bn1.weight
layer3.0.bn1.bias
layer3.0.conv2.weight
layer3.0.bn2.weight
layer3.0.bn2.bias
layer3.0.downsample.0.weight
layer3.0.downsample.1.weight
layer3.0.downsample.1.bias
layer3.1.conv1.weight
layer3.1.bn1.weight
layer3.1.bn1.bias
layer3.1.conv2.weight
layer3.1.bn2.weight
layer3.1.bn2.bias
layer4.0.conv1.weight
layer4.0.bn1.weight
layer4.0.bn1.bias
layer4.0.conv2.we

In [10]:
finetune_net=torchvision.models.resnet18(pretrained=True)
finetune_net.fc=nn.Linear(finetune_net.fc.in_features,2)
nn.init.xavier_uniform_(finetune_net.fc.weight)

Parameter containing:
tensor([[-0.0130,  0.0993, -0.0658,  ..., -0.0859, -0.0830,  0.0915],
        [ 0.0081,  0.0690,  0.0885,  ..., -0.0378,  0.1051,  0.1031]],
       requires_grad=True)

In [12]:
def evaluate_acc_gpu(net,data_iter,device):
    if isinstance(net, nn.Module):
        net.to(device)
        net.eval()
        l_sum=[]
        with torch.no_grad():
            for X,y in data_iter:
                X,y=X.to(device),y.to(device)
                y_hat=net(X)
                l_sum.append((sum(torch.argmax(y_hat,dim=1).reshape(y.shape)==y)/y.shape[0]).item())
        return sum(l_sum)/len(l_sum)
    

In [13]:
def train_ch66(net,train_iter,test_iter,loss,trainer,num_epochs,device):
#     def init_weight(m):
#         if type(m)==nn.Linear or type(m)==nn.Conv2d:
#             nn.init.xavier_uniform_(m.weight)
#     net.apply(init_weight)
    print('train on : ',device)
    net.to(device)
    loss=loss
    optimizer=trainer
    for epoch in range(num_epochs):
        tr_l,tr_acc=[],[]
        net.train()
        for X,y in train_iter:
            X,y=X.to(device),y.to(device)
            y_hat=net(X)
            l=loss(y_hat,y).sum()
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            
            with torch.no_grad():
                tr_l.append(l.item())
                tr_acc.append((sum(torch.argmax(y_hat,dim=1).reshape(y.shape)==y)/y.shape[0]).item())
        test_acc=evaluate_acc_gpu(net,test_iter,device)
        print('epoch : ',epoch+1,' train loss : ',sum(tr_l)/len(tr_l),' train acc : ',sum(tr_acc)/len(tr_acc),' test acc : ',test_acc)
            

In [16]:
def train_fine_tuning(net,lr,batch_size=128,num_epochs=5,param_group=True):
    device='cuda' if torch.cuda.is_available() else 'cpu'
    train_iter=torch.utils.data.DataLoader(torchvision.datasets.ImageFolder(r'F:\study\ml\HotDog\train',transform=train_augs),
                                          batch_size=batch_size,shuffle=True)
    test_iter=torch.utils.data.DataLoader(torchvision.datasets.ImageFolder(r'F:\study\ml\HotDog\test',transform=test_augs),
                                          batch_size=batch_size,shuffle=True)
    loss=nn.CrossEntropyLoss(reduction='none')
    if param_group:
        params_1x=[p for n,p in net.named_parameters() if n not in ["fc.weight","fc.bias"]]
        trainer=torch.optim.SGD([{'params':params_1x},
                                {'params':net.fc.parameters(),'lr':lr*10}],
                               lr=lr,weight_decay=0.001)
    else:
        trainer=torch.optim.SGD(net.parameters(),lr=lr,weight_decay=0.001)
    train_ch66(net,train_iter,test_iter,loss,trainer,num_epochs,device)

In [17]:
train_fine_tuning(finetune_net,5e-5)

train on :  cpu
epoch :  1  train loss :  243.84019720554352  train acc :  0.6927734389901161  test acc :  0.9073660714285714
epoch :  2  train loss :  36.0447438955307  train acc :  0.8987304680049419  test acc :  0.9263392857142857
epoch :  3  train loss :  31.070427179336548  train acc :  0.9092773422598839  test acc :  0.8839285714285714
epoch :  4  train loss :  24.14473992586136  train acc :  0.9327148422598839  test acc :  0.9508928571428571
epoch :  5  train loss :  28.214502960443497  train acc :  0.9170898422598839  test acc :  0.9475446428571429


In [18]:
finetune_net=torchvision.models.resnet18(pretrained=True)
finetune_net.fc=nn.Linear(finetune_net.fc.in_features,2)
nn.init.xavier_uniform_(finetune_net.fc.weight)

Parameter containing:
tensor([[ 0.1019,  0.0707, -0.0856,  ..., -0.0540, -0.0223, -0.0031],
        [ 0.1012,  0.0827, -0.0777,  ...,  0.0032, -0.0235, -0.0919]],
       requires_grad=True)

In [19]:
pretrained_net=torchvision.models.resnet18(pretrained=True)

In [20]:
weight=pretrained_net.fc.weight
hotdog_w=torch.split(weight,1,dim=0)[713]

In [22]:
with torch.no_grad():
    finetune_net.fc.weight.data[0]=hotdog_w.reshape(-1)

In [ ]:
train_fine_tuning(finetune_net,5e-5)

train on :  cpu
epoch :  1  train loss :  302.06534075737  train acc :  0.6920898444950581  test acc :  0.9095982142857143
epoch :  2  train loss :  36.60175085067749  train acc :  0.9041015617549419  test acc :  0.9408482142857143
